<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V0_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [73]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [74]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
#print(Hardwood_Sawlog_Data)

**SAWMILL CLASS**

In [76]:
#SawMill Object
class Mill:
  m = GEKKO(remote=True)

  def __init__(self, Id, Wood, Prod, Dis, Dem=0, Radio = 20):
    self.Id = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    self.Dist = Dis
    self.Demand = Dem
    self.Radio = Radio


  def Harvest_Close(self, Available, Dem = 0):
    if (Dem>0):
      self.Demand = Dem
    
    Num = len(self.Dist)
  
    H = m.Array(m.Var,(Num),value = 0)

    for i in range(Num):
      H[i].lower = 0
      H[i].upper = int(Available[i])
      
    prod0 = m.Intermediate(m.sum([self.Dist.loc[i,('DIST')]*H[i] for i in range(0,len(H))]))
    
    m.Equation(self.Demand==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=True)
    a = self.Dist
    flat = [item for l in H for item in l]
    a['HARVESTED'] = flat 
    return(a)
    

In [77]:
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av)
    self.Dist = Dist
    self.Mills = []


  def Add_Mill2(self, Id, Wood, Prod, Dem=0):

    a =self.Cell_Less(Id,0,20)
    D2 = a.loc[:, ('DEST','DIST')]
    mil= Mill(id,Wood,Prod,D2,Dem)
    self.Mills.append(mil)
  
  def Harvest_Mill(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    ans = self.Mills[j].Harvest_Close(self.Available[t].SumOfTotal,self.Mills[j].Demand)
    print("SawMill ",j,ans)
    #print("Tipo de variable Available", type(self.Available[0]))
    #print("Available\r",self.Available)
    
    for j in range(len(ans)):                                                                         
      if not(self.Available[0][self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[0]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        #print("Available in position a2:\r",self.Available[0].loc[a2,("SumOfTotal")])
        self.Available[0].loc[a2,("SumOfTotal")] = self.Available[0].loc[a2,("SumOfTotal")] - ans.loc[j,('HARVESTED')]
      else:
        print('Elemento no Hallado: ',ans.loc[j,('DEST')])
   
  
  def Harvest_All_Mills(self):
    for k in range(len(self.Mills)):
      Hardwood_Sawlog.Harvest_Mill(0,k)
    
  
  def Cell_Less(self, Id_Mill, Time, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))

    Ans["AVAIL"] = Disp 
    return(Ans)
  

  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    #print("Numero de celdas ",len(Ans))
    return(len(Ans))
    
  




**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [78]:
# Main Program
start_time = time.time()
#Num_Cell= 10000
#D = np.zeros(Num_Cell)
#Demand = 20000000 

#for i in range(Num_Cell):
#  D[i] = i

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
#print("Hardwood Negativo",Hardwood_Sawlog_Data[Hardwood_Sawlog_Data['SumOfTotal']==0])
#a =Hardwood_Sawlog_Data.duplicated()
#print(a.describe())
#print(Hardwood_Sawlog.Available[0]["Unique_ID"]==74031)

#Hardwood_Sawlog.Cell_Less(54549,0,10)
ma = Dist_Data['ORIG'].unique()
#print('Ma', ma[0])
#print(Hardwood_Sawlog_Data)

#a = Mill(74031,0,0,Hardwood_Sawlog.Dist.DIST,1000000)
#a = Mill(Mills_Data['Unique_ID'][0],Mills_Data['plant'][0],Mills_Data['s_h'][0],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][0])
#a.Harvest_Close(Hardwood_Sawlog_Data.SumOfTotal, 10, 1000000)
#print(Hardwood_Sawlog.Dist.loc[:, ('DIST')])

#Mills_Data.loc[0, ('Unique_ID')]

for k in range(1):
  
  #Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))
  #mi= Mill(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Hardwood_Sawlog.Dist.loc[:, ('DIST')],Mills_Data.loc[k,('Demand')])
  #Hardwood_Sawlog.Add_Mill(mi)
  #Hardwood_Sawlog.Add_Mill2(Mills_Data.loc[k, ('Unique_ID')],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
  Hardwood_Sawlog.Add_Mill2(ma[k],Mills_Data.loc[k,('plant')],Mills_Data.loc[k,('s_h')],Mills_Data.loc[k,('Demand')])
  #print("Cosechando Aserradero ", k)
  #Hardwood_Sawlog.Harvest_Mill(0,k)

Hardwood_Sawlog.Harvest_All_Mills()
print ("My program took", time.time() - start_time, "to run")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


SawMill  0       DEST       DIST     HARVESTED
0    54549   0.000000  6.800000e+04
1    54749   2.829784  9.951995e-08
2    54150   2.938933  9.651872e-08
3    54949   3.067795  9.086942e-08
4    54748   3.524870  7.800959e-08
..     ...        ...           ...
166  52352  19.726610  5.677814e-09
167  55759  19.762639  5.664160e-09
168  56156  19.772028  5.646188e-09
169  54953  19.828987  5.596670e-09
170  55347  19.982425  5.478057e-09

[171 rows x 3 columns]
 2287844.7
 2268199.6119999997
 2280210.589
 2287724.078
 2294929.5
 2196091.636
 2294260.518
 2155774.259
Elemento no Hallado:  53950
 2298647.5
 2295427.991
 2279530.063
 2219389.31
Elemento no Hallado:  54149
 2293918.667
 2287595.086
Elemento no Hallado:  54151
Elemento no Hallado:  53749
 2300877.9080000003
Elemento no Hallado:  53550
 2173385.607
 2362789.518
 2314771.143
 2148396.625
Elemento no Hallado:  54348
Elemento no Hallado:  53750
 2278208.339
 2270021.121
 2208712.379
Elemento no Hallado:  53951
Elemento no Hall